In [19]:
import pandas as pd
import numpy as np

# JIRA libraries
from jira.resources import IssueLink
import jira.client
from jira.client import JIRA
from jira import JIRA
import re

ModuleNotFoundError: No module named 'jira'

In [17]:
# import the deploy excel file into a dataframe
df_deploy = pd.read_excel('output.xlsx', sheet_name = 0)
df_deploy

,Node,Junction Box,Port,Communications,Cable to Connector Panel,Cable EXT ID,IP Address,Instrument Category,Instrument,Serial Number,Device ID,Component,Work Ticket,Linked To,Operation
0,Cascadia,"JB-13, 2700 m","J3 (15V, 75W)","Serial, RS485",1048181-XX,Ext 429,10.136.68.130:4002,CTD,SBE 19 Plus V2,7033.0,23580,Test and Development,https://jira.oceannetworks.ca/browse/EN-3609,NaN,Deploy
1,Cascadia,"JB-13, 2700 m","J3 (15V, 75W)","Serial, RS485",1048181-XX,Ext 429,10.136.68.130:4002,DO,SBE 63 DO,7033.0,23580,Test and Development,EN-3610,NaN,Deploy
2,Cascadia,"JB-13, 2700 m","J5 (48V, 75W)",Serial,1048181-XX,"1093189-37 (EXT-xx), ODI-12-Receptacle to XSL-...",10.136.68.130:4003,ADCP 75 kHz,RDI Workhorse Long Ranger,16511.0,23006,Test and Development,https://jira.oceannetworks.ca/browse/EN-3580,NaN,Deploy
3,Cascadia,"JB-13, 2700 m","J10 (400V, 15A)",Ethernet,1048181-XX,TBD,NaN,BPR,"NRCan Bottom Pressure Recorder, SE",87.0,21502,Test and Development,https://jira.oceannetworks.ca/browse/EN-3826,NaN,Deploy
4,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone Array,icLink,NaN,50320,Test and Development,https://jira.oceannetworks.ca/browse/EN-3926,NaN,Deploy
5,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320,Test and Development,NaN,NaN,Deploy
6,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320,Test and Development,NaN,NaN,Deploy
7,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320,Test and Development,NaN,NaN,Deploy
8,Cascadia CORK (quiet),"JB-18, 2660 m","J8 (400V, 15A)",Ethernet,"1048181-85, EXT-809","1093189-37, EXT-xx ,70m -> connects to EXT-xx",NaN,Hydrophone,IcListen HF,NaN,50320,Test and Development,NaN,NaN,Deploy
9,Clayoquot,JB-08,J10 (400V 15A),Ethernet,1048181-xx,"1093189-xx, EXT-xx ,70m (already on seafloor) ...",NaN,Hydrophone Array,icLink,4042.0,48940,Test and Development,https://jira.oceannetworks.ca/browse/EN-3924,NaN,Deploy


In [18]:
# Connect to jira
# Authentication done by using username and password
username = 'mtcelec2'
password = '1q2w3e4R!'

jira = JIRA(
    basic_auth = (username, password),
    options = {'server': 'http://142.104.193.65:8080'}
    #options = {'server': 'https://jira.oceannetworks.ca/'}
)

# initialzie the variable
__instrument = ''
__serialNumber = ''
__deviceID = ''

# Descriptions
__location = ''
__communications = ''
__cableID = ''
__instrumentCategory = ''
__ipAddress = ''
__junctionBox = ''
__port = ''


__components = ''
__outwardIssue = ''
__summaryTitle = 'Instrument Qualification'

# recod the current partent EN number
currEN = ''


NameError: name 'JIRA' is not defined

In [14]:
# get the first value of the port column
# for comparing if it is a partent or a child
currPort = df_deploy.iloc[0]['Port']
meetInitialPort = False

for index, row in df_deploy.iterrows():
    if(row['Port'] != currPort):
        # this means this row is a new parent row
        # check wether it has a ticket already
        # if yes, then leave it alone, and update the currEN
        # if no, then create a new ticket, write the URL to the dataframe, update the currEN
        if(isinstance(row['Work Ticket'], str)):
            # Then it has a ticket
            # Use regular expression to record the EN number
            findEN = 'EN'
            currEN = row['Work Ticket'][row['Work Ticket'].find(findEN):]
        else:
            # Then it does not have a ticket
            # create a ticket, write the URL to dataframe and update the currEN
            
            # Assign the variables
            __instrument = row['Instrument']
            __serialNumber = row['Serial Number']
            __deviceID = row['Device ID']

            __location = row['Node']
            __communications = row['Communications']
            __cableID = row['Cable']
            __instrumentCategory = row['Instrument Category']
            __ipAddress = row['IP Address']

            __components = row['Component']
            __junctionBox = row['Junction Box']
            __port = row['Port']
            
            #create a ticket for current row
            new_issue = jira.create_issue(
                project = {'key': 'EN'}, 
                summary = "%s: %s SI: %s DI: %s" % (__summaryTitle, __instrument, __serialNumber, __deviceID),
                description = "Site Location: %s\n Communications: %s\n Cable EXT ID: %s\n Instrument Category: %s\n IP Address: %s\n Junction Box: %s\n Port: %s\n" % (__location, __communications, __cableID, __instrumentCategory, __ipAddress, __junctionBox, __port), 
                issuetype = {'name': 'Task'}, 
                components = [{'name' : __components}],
                customfield_10794 = {'id': "10453"},            # Bill of work to Customers (Default: ONC Internal)
                customfield_10592 = "%s" % __serialNumber,      # Serial # field
                customfield_10070 = __deviceID)                 # Device ID field
            
            # write the new ticket URL into dataframe
            row['Work Ticket'] = "http://142.104.193.65:8080/browse/%s" % new_issue.key
            
            # update EN number
            currEN = new_issue.key
                
    if(row['Port'] == currPort and meetInitialPort == True):
        # This is a child row
        # check whether has the ticket already
        # if yes, update the ticket with linkedto 
        # if no, fill the linkedto with currEN and create a new ticket and write into dataframe
        if(isinstance(row['Work Ticket'], str)):
            # Then it has a ticket
            # Use regular expression to record the EN number
            findEN = 'EN'
            childEN = row['Work Ticket'][row['Work Ticket'].find(findEN):]
            
            # fill the linkedTo with currEN
            row['Linked To'] = currEN
            
            # update the jira ticket
            # assign the variables
            __instrument = row['Instrument']
            __serialNumber = row['Serial Number']
            __deviceID = row['Device ID']

            __location = row['Node']
            __communications = row['Communications']
            __cableID = row['Cable']
            __instrumentCategory = row['Instrument Category']
            __ipAddress = row['IP Address']

            __components = row['Component']
            __junctionBox = row['Junction Box']
            __port = row['Port']
            __outwardIssue = currEN
            
            # Issue to be updated
            issue = jira.issue(childEN)
            
            # update selected issue
            issue.update(
                project = {'key': 'EN'}, 
                summary = "%s: %s %s %s" % (__summaryTitle, __instrument, __serialNumber, __deviceID),
                description = "Site Location: %s\n Communications: %s\n Cable EXT ID: %s\n Instrument Category: %s\n IP Address: %s\n Junction Box: %s\n Port: %s\n" % (__location, __communications, __cableID, __instrumentCategory, __ipAddress, __junctionBox, __port), 
                customfield_10794 = {'id': "10453"},            # Bill of worl to Customers (Default: ONC Internal)
                customfield_10592 = "%s" % __serialNumber,      # Serial # field (Should be string)
                customfield_10070 = __deviceID)                 # Device ID field

            # update the linkedto
            jira.create_issue_link("Related", issue, currEN, None)
            
        else:
            # Then it does not have a ticket
            # fill the linkedto cell with currEN and create a new ticket and write the URL to dataframe
            
            # fill the linkedTo with currEN
            row['Linked To'] = currEN
            
            # create a new jira ticket
            # assign the values
             __instrument = row['Instrument']
            __serialNumber = row['Serial Number']
            __deviceID = row['Device ID']

            __location = row['Node']
            __communications = row['Communications']
            __cableID = row['Cable']
            __instrumentCategory = row['Instrument Category']
            __ipAddress = row['IP Address']

            __components = row['Component']
            __junctionBox = row['Junction Box']
            __port = row['Port']
            __outwardIssue = currEN
            
            #create a ticket for current row
            new_issue = jira.create_issue(
                project = {'key': 'EN'}, 
                summary = "%s: %s SI: %s DI: %s" % (__summaryTitle, __instrument, __serialNumber, __deviceID),
                description = "Site Location: %s\n Communications: %s\n Cable EXT ID: %s\n Instrument Category: %s\n IP Address: %s\n Junction Box: %s\n Port: %s\n" % (__location, __communications, __cableID, __instrumentCategory, __ipAddress, __junctionBox, __port), 
                issuetype = {'name': 'Task'}, 
                components = [{'name' : __components}],
                customfield_10794 = {'id': "10453"},            # Bill of work to Customers (Default: ONC Internal)
                customfield_10592 = "%s" % __serialNumber,      # Serial # field
                customfield_10070 = __deviceID)                 # Device ID field
            
            # add the linkedto
            jira.create_issue_link("Related", new_issue.key, currEN, None)
            
            # write the new ticket URL into dataframe
            row['Work Ticket'] = "http://142.104.193.65:8080/browse/%s" % new_issue.key
            
            # do not need to update the EN number because this is a child ticket
            
        
    # This is row one
    if(row['Port'] == currPort and meetInitialPort == False):
        # Mark as already seen the first row
        meetInitialPort = True
        
        # Check whether it has the ticket already
        # if yes, record the EN number
        # if not, create the ticket and record the EN number
        if(isinstance(row['Work Ticket'], str)):
            # Then it has the ticket
            # Use regular expression to record the EN number
            findEN = 'EN'
            currEN = row['Work Ticket'][row['Work Ticket'].find(findEN):]
        else:
            # Then it has no ticket
            # create a new ticket, write into the dataframe and record the EN number
            
            # assign the variables
            __instrument = row['Instrument']
            __serialNumber = row['Serial Number']
            __deviceID = row['Device ID']

            __location = row['Node']
            __communications = row['Communications']
            __cableID = row['Cable']
            __instrumentCategory = row['Instrument Category']
            __ipAddress = row['IP Address']

            __components = row['Component']
            __junctionBox = row['Junction Box']
            __port = row['Port']
            
            #create a ticket for current row
            new_issue = jira.create_issue(
                project = {'key': 'EN'}, 
                summary = "%s: %s SI: %s DI: %s" % (__summaryTitle, __instrument, __serialNumber, __deviceID),
                description = "Site Location: %s\n Communications: %s\n Cable EXT ID: %s\n Instrument Category: %s\n IP Address: %s\n Junction Box: %s\n Port: %s\n" % (__location, __communications, __cableID, __instrumentCategory, __ipAddress, __junctionBox, __port), 
                issuetype = {'name': 'Task'}, 
                components = [{'name' : __components}],
                customfield_10794 = {'id': "10453"},            # Bill of work to Customers (Default: ONC Internal)
                customfield_10592 = "%s" % __serialNumber,      # Serial # field
                customfield_10070 = __deviceID)                 # Device ID field
            
            # write the new ticket URL into dataframe
            row['Work Ticket'] = "http://142.104.193.65:8080/browse/%s" % new_issue.key
            
            # record EN number
            currEN = new_issue.key
            

            
            
            
        
    
    

'J3 (15V, 75W)'